In [1]:
import tensorflow as tf
from keras.datasets import mnist

2023-08-12 10:42:29.212303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_visible_devices([], 'GPU')

In [8]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [3]:
rows = 28
cols = 28
channels = 1
image_size = (rows,cols,channels)

In [4]:
# For an input noise or latent vector generate a fake image

def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)       

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(image_size), activation='tanh'))
    model.add(Reshape(image_size))

    model.summary()

    noise = Input(shape=noise_shape)
    generated_img = model(noise)    #Generated image

    return Model(noise, generated_img)

In [5]:
# given a fake image tell me the likelihood if it is real or fake

def build_discriminator():
    model = Sequential()

    model.add(Flatten(input_shape=image_size))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=image_size)
    validity = model(img)

    return Model(img, validity)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
def save_imgs(epoch):
    # print 5x5=25 generated images
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [ ]:
"""
Steps:
Train the discriminator:
        we use gen and disc separately:
                generate fake half batch size images from generator
                ask discriminator for loss on these half real and half fake
                average the loss
Train the generator:
        we use both the networks together, combined:
                generate fake full batch size images from generator
                pass it to the network saying the discriminator that these fake images are real (this is the way we hold disc constant while training gen)
"""

def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

    X_train = np.expand_dims(X_train, axis=3) # make from 28x28 to 28x28x1

    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):
        #  Train Discriminator

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

        # noise for the generator
        # half_batch 1: rnd,rnd,rnd ... 100
        # half batch 2: rnd,rnd,rnd ... 100 and so on
        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images by the generator first
        gen_imgs = generator.predict(noise)

        # ask discriminator for loss test on both real and fake images
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        
        # average the loss
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

        #  Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100)) 

        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        g_loss = combined.train_on_batch(noise, valid_y)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)